# Код для извлечения чистого текста из html-кода с сохранением его структуры (абзацы, заголовки).

Этот код позволяет извлекать текст из html кода с пометками соответствующих абзацев. Список абзацев можно найти в массивах headings и paragraphs. Под каждый из массивов есть условный знак, с которым хранится абзац соответствующего типа. При желании его можно разнообразить.

In [17]:
from bs4 import BeautifulSoup


sample_html_code = '''
<html>
  <body>
    <h1>This is a heading</h1>
    <p>This is a paragraph.</p>
    <p>Another paragraph.</p>
  </body>
</html>
'''

def MarkText(html_code : str):

    soup = BeautifulSoup(html_code, 'html.parser')

    headings = ['h1', 'h2', 'h3', 'h4', 'h5', 'h6']
    paragraphs = ['p', 'pre']

    HEADING_MARK = '//@'
    PARAGRAPH_MARK = '//&'

    def process_tag(tag):
        if tag.name in headings:
            return HEADING_MARK + tag.text.strip() + '\n'
        elif tag.name in paragraphs:
            return PARAGRAPH_MARK + tag.text.strip() + '\n'
        else:
            return ''


    result = ''

    for tag in soup.find_all():
        result += process_tag(tag)

    return result
print(MarkText(sample_html_code))

//@This is a heading
//&This is a paragraph.
//&Another paragraph.



# Код для сохранения иерархической связи между страницами веб-сайтов с использованием парсера Морозова

Здесь мы подключаемся к удалённой реляционной базе данных, создаём базу pages с полями, соответствующими логике ниже и создаём курсор для итерации по базе данных.

CREATE TABLE Pages (

    PageID INT PRIMARY KEY,
    ParentPageID INT,
    PageLevel INT
    
)

In [19]:
import sqlite3

conn = sqlite3.connect('history.db')
cur = conn.cursor()

cur.execute('''CREATE TABLE Pages (
    PageID INT PRIMARY KEY,
    ParentPageID INT,
    PageLevel INT
)''')

Мы пока что не получили готовый парсер от ребят, поэтому вставили наш код в парсер Морозова, делавшего его раньше. Для этого мы использовали словарь вида ссылка:id_страницы. Вставляем строки с очередным сайтом в иерархическую таблицу. Также была переписана рекурсивная часть обработки ссылок Морозова на алгоритм поиск в ширину, чтобы более корректно выставлять уровни важных ссылок. Мы не смогли его протестировать, потому что таблица raex_list, из которой брались данные, видимо хранилась у Морозова локально.

In [23]:
url_id = {}
cur.execute("SELECT COUNT(*)")
first_id = cur.fetchone()[0] + 1
def parsing_site(company, url, depth, max_depth=0):
#     print(url)
    cur.execute("INSERT INTO Pages (ParentPageID, PageLevel) VALUES (?, ?)", (-1, 0))
    url_id[url] = first_id
    first_id += 1
    internalLinks = [(url, '/', 0)] #  Создаем массив ссылок, которые обрабатываем обходом в ширину.
    used = []
    path_from = {}
    while internalLinks:
        #  new
        now = internalLinks.pop(0)
        url = now[0]
        from_url = now[1]
        depth = now[2]
        #  Помечаем ссылку как использованную, чтоб не зациклиться.
        used.append(url)
        # Сохраняем ссылку откуда пришли, чтобы потом восстановить путь до корня.
        path_from[url] = from_url
        #  new

        response = requests.get(url=url, timeout=timeout, headers=headers, verify=False)
        soup = BeautifulSoup(response.content, 'lxml')

        for script in soup(["script", "style"]):
            script.extract()

        text = soup.get_text()

        lines = (line.strip() for line in text.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        text = '\n'.join(chunk for chunk in chunks if chunk).replace('-\n', '').lower()
        text = re.sub(r'[`!@#$%^&*()_+\-=\[\]{};\':"\\|,.<>\/?~©«»—]', r'', text)
        text = ''.join([i for i in text if not i.isdigit()])

        text = list(np.concatenate([sent_tokenize(i.strip()) for i in text.split('\n')]).flat)

        text_lemmatized = []
        for line in [t.split() for t in text]:
            line_lemmatized = ' '.join([morph.normal_forms(l)[0] for l in line if l not in stopwords_ru])
            text_lemmatized.append(line_lemmatized)

        spans = list(filter(None, [span.string for span in soup.find_all('span')]))

        if len(spans):
            spans = '\n'.join(span for span in spans if span).replace('-\n', '').lower()
            spans = re.sub(r'[`!@#$%^&*()_+\-=\[\]{};\':"\\|,.<>\/?~©«»—]', r'', spans)
            spans = ''.join([i for i in spans if not i.isdigit()])

            spans = list(np.concatenate([sent_tokenize(i.strip()) for i in spans.split('\n')]).flat)
        else:
            spans = []

        spans_lemmatized = []
        for line in [s.split() for s in spans]:
            line_lemmatized = ' '.join([morph.normal_forms(l)[0] for l in line if l not in stopwords_ru])
            spans_lemmatized.append(line_lemmatized)

        df1 = pd.DataFrame(text_lemmatized, columns=['text'])
        df2 = pd.DataFrame(spans_lemmatized, columns=['text'])
        data = pd.concat([df1, df2]).drop_duplicates().reset_index(drop=True)
        data['company'] = company['Название']
        data['rating'] = company['№']
        data['url'] = url
        data = data[['rating', 'company', 'url', 'text']]

        # Считываем все ссылки на сайте и добавляем их в конец нашей очереди обхода в ширину.
        # Проверяем так же глубину обхода - слишком далеко нам уходить не надо.
        if depth < max_depth:
            NewLinks = [
                a.get('href') for a in soup.find_all('a')
                if a.get('href') and a.get('href').startswith('/')
            ]
            NewLinks = [link[1:] for link in NewLinks]
            NewLinks = [*set(NewLinks)]
            NewLinks = list(filter(None, NewLinks))

            for link in NewLinks:
                    if ".pdf" not in link and ".PDF" not in link:
                        new_url = os.path.join(url[:url.rfind('www')], urlparse(url).netloc, link)
                        #Тут как раз добавляем URL в конец очереди.
                        if new_url not in used:
                            internalLinks.append((new_url, url, depth + 1))
                            cur.execute("INSERT INTO Pages (ParentPageID, PageLevel) VALUES (?, ?)", (url_id[url], depth + 1))
                            url_id[new_url] = first_id
                            first_id += 1


        #  Здесь вставляем наш PAGE_ID
        #  Путь до корневой папки и будет иерархической определяющей нашей страницы.
        #  Восстанавливается он следующим образом :
        path_now = []
        while url != '/' :
            path_now.append(url)
            url = path_from[url]
        path_now.reverse()

        # В path_now хранится путь до страницы из корня!
    return data